In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from multiprocessing import Pool
import time
from tqdm.notebook import tqdm
import missingno as msno
from utils import get_idle_proportion



In [2]:
"""" 
We would like to set up a ML problem. 

In this problem we would like to predict the proportion of being idle for the next interval for a given node.
This makes the problem a time series problem with regression falvour as we predict a continues value
between 0 and 1.  


Another setup would be to turn this into a classification problem by cutting the interval [0-1] into multiple classes

class low idle: [0-0.3]
class middle idle: [0.3-0.9]
class high idle: [0.9-1]

Then we predict the next step belongs to which class. This is more realistic because differenticating between 
0.91 and 0.92 does not have much value(right?)
"""

'" \nWe would like to set up a ML problem. \n\nIn this problem we would like to predict the proportion of being idle for the next interval for a given node.\nThis makes the problem a time series problem with regression falvour as we predict a continues value\nbetween 0 and 1.  \n\n\nAnother setup would be to turn this into a classification problem by cutting the interval [0-1] into multiple classes\n\nclass low idle: [0-0.3]\nclass middle idle: [0.3-0.9]\nclass high idle: [0.9-1]\n\nThen we predict the next step belongs to which class. This is more realistic because differenticating between \n0.91 and 0.92 does not have much value(right?)\n'

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.precision = 4 # show 4 digits precision
folder_path_slurm_data = Path('./system_analytics_2024/slurm_data')
folder_path_prom_data = Path('./system_analytics_2024/prom_data')
folder_path_saving_results = Path('./results')

file_path_sinfo_enriched = folder_path_prom_data/'cleaned_data_for_ML'/'sinfo_enriched.parquet.gzip'
df = pd.read_parquet(file_path_sinfo_enriched)

df.sample(n=10)

,node,time_3hour_interval,idle_duration,all_state_durations_in_interval,idle_proportion,last_state,time_for_last_state,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds_min,node_boot_time_seconds_max,node_boot_time_seconds_mean,node_boot_time_seconds_median,node_context_switches_total_min,node_context_switches_total_max,node_context_switches_total_mean,node_context_switches_total_median,node_cooling_device_cur_state_min,node_cooling_device_cur_state_max,node_cooling_device_cur_state_mean,node_cooling_device_cur_state_median,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total_min,node_cpu_core_throttles_total_max,node_cpu_core_throttles_total_mean,node_cpu_core_throttles_total_median,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz_min,node_cpu_frequency_max_hertz_max,node_cpu_frequency_max_hertz_mean,node_cpu_frequency_max_hertz_median,node_cpu_frequency_min_hertz_min,node_cpu_frequency_min_hertz_max,node_cpu_frequency_min_hertz_mean,node_cpu_frequency_min_hertz_median,node_cpu_package_throttles_total_min,node_cpu_package_throttles_total_max,node_cpu_package_throttles_total_mean,node_cpu_package_throttles_total_median,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total_min,node_forks_total_max,node_forks_total_mean,node_forks_total_median,node_hwmon_power_average_interval_max_seconds_min,node_hwmon_power_average_interval_max_seconds_max,node_hwmon_power_average_interval_max_seconds_mean,node_hwmon_power_average_interval_max_seconds_median,node_hwmon_power_average_interval_min_seconds_min,node_hwmon_power_average_interval_min_seconds_max,node_hwmon_power_average_interval_min_seconds_mean,node_hwmon_power_average_interval_min_seconds_median,node_hwmon_power_average_interval_seconds_min,node_hwmon_power_average_interval_seconds_max,node_hwmon_power_average_interval_seconds_mean,node_hwmon_power_average_interval_seconds_median,node_hwmon_power_average_watt_min,node_hwmon_power_average_watt_max,node_hwmon_power_average_watt_mean,node_hwmon_power_average_watt_median,node_hwmon_power_is_battery_watt_min,node_hwmon_power_is_battery_watt_max,node_hwmon_power_is_battery_watt_mean,node_hwmon_power_is_battery_watt_median,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total_min,node_intr_total_max,node_intr_total_mean,node_intr_total_median,node_load1_min,node_load1_max,node_load1_mean,node_load1_median,node_load15_min,node_load15_max,node_load15_mean,node_load15_median,node_load

In [ ]:
# do a simple point prediction just based on the past signals and the timing.
# add the last state as well and see how things improve.
# then get the number of jobs running and the last moment jobs


# Do the same thing with the mind set that we only predict if it will be 100 idle or not.
# this becomes a binary classficiation, but still it is a time series as you have
# respet the temoral of the signals.